<a href="https://colab.research.google.com/github/ViacheslavCRBP/pythonProject/blob/master/HW_13_GPT_ViacheslavS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Генерация из GPT

Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза


In [1]:
!pip install transformers

In [2]:
!wget https://huggingface.co/datasets/merionum/ru_paraphraser

--2024-01-26 07:55:17--  https://huggingface.co/datasets/merionum/ru_paraphraser
Resolving huggingface.co (huggingface.co)... 18.155.173.45, 18.155.173.126, 18.155.173.64, ...
Connecting to huggingface.co (huggingface.co)|18.155.173.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314818 (307K) [text/html]
Saving to: ‘ru_paraphraser.2’

ru_paraphraser.2    100%[===================>] 307.44K  --.-KB/s    in 0.1s    

2024-01-26 07:55:17 (2.70 MB/s) - ‘ru_paraphraser.2’ saved [314818/314818]



In [3]:
with open('tweets.txt', 'r') as f:
    tweets = f.read().strip().split('\n\n')
print(len(tweets))
for i in range(3):
    print(tweets[i])

26
Соловьев наконец-то вышел на новый уровень - теперь его стали банить и в офлайне
Дарим мы тебе бутылку игристого вина. Пить тебе еще рано, но встретиться с ней за некоторые преступления ты уже можешь. ПОЗ-ДРАВ-ЛЯ-ЕМ!
Да. Еще очень многие помнят, что такое госплан, как планировалось, талоны на еду, очереди, дефицит, выездные визы. Но спасибо, что напомнил


In [4]:
import torch
device = torch.device("cuda")

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
import random

In [8]:
sep = '\n***\n'

prefix = sep.join([''] + random.sample(tweets, k=5) + [''])

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]
print(prefix)


***
Внимание. В России официально появился парк славы Илона Маска
***
Тонер, картриджи и много-много бумаги
***
И немножечко зиганул
***
Да. Еще очень многие помнят, что такое госплан, как планировалось, талоны на еду, очереди, дефицит, выездные визы. Но спасибо, что напомнил
***
- Скажи что-нибудь сладкое?
- Сахар
- Не. Еще слаще
- Мёд
- ЕЩЕ СЛАЩЕ!!!
- Бюджет!!!
- Вооооот
***



In [9]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens,
    #end_token=end_token_id,
    do_sample=False,
    max_length=size+128,
    repetition_penalty=4.2,
    temperature=0.7,
    num_beams=3,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


А еще у нас в городе есть улица имени космонавта Леонова!
***
Я не знаю, кто это такой, но он был первым человеком, ступившим на поверхность Луны...
***
У меня сегодня день рождения - мне исполнился 21 год!
***
С утра я проснулась с мыслью о том, что надо бы сходить к стоматологу. А потом вспомнила, что уже давно ничего не чистила.<s>
Прогулки по Москве 

1. На ВДНХ:

2. У стен Кремля:

3. Памятник Минину и Пожарскому:

4. Храм Христа


# Диалоги

Логика та же самая, что и с твитами. Только выбрали разделитель в стиле bash.org

In [10]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        eos_token_id=end_token_id,
        do_sample=True,
        max_length=size+128,
        repetition_penalty=3.2,
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [ ]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
Что же нам теперь делать?
А что мы можем сделать?
Муравью сможем хвост на лоб приделать?
Нет, не сможем.
Давай Аляску заберем?
Не-а! Муравей - он и в Африке муравей.
Давай Аляску и Африку заберем?
Ага! А еще лучше Антарктиду...
Отличная идея!
Ну а дальше что?
А дальше Аляску заберем...
С чего начнем?
С Анкорджа
Это как?
Запустим туда мигрантов
Как это?
на лодках
Зачем?
чтобы забрать Аляску, запустим туда мигрантов на лодках
И все?
Тупишь?


In [ ]:
for i in range(10):
    print(respond_to_dialog(['Давай поговорим о домашних животных', 'Каких питомцев вы любите?\n--------',
                             'Давай поговорим о машинах', 'Какого цвета твой автомобиль?\n--------',
                             'Давай поговорим о физике']))